# Проверка точности и адекватности модели, отвечающей на запросы о растениях, а также для выявления проблем в ее работе. Точность модели или accuracy

### Импорт библиотек

In [ ]:
import pandas as pd
#!pip install time
#!python -m pip install --upgrade pip
#!pip install pandas python-telegram-bot fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import fuzz
from collections import defaultdict


In [ ]:
route = r"C:\Data_sets\data-60861-2024-08-06.csv" # прочитаем файл
df = pd.read_csv(route, sep=';', encoding='utf-8', on_bad_lines='skip')

df.head()

,ID,Name,LatinName,Photo,LandscapingZone,ProsperityPeriod,Description,LocationPlace,ViewForm,global_id,Unnamed: 10
0,Код,Название,Латинское название,Фотография,Ландшафтная зона,Период цветения,Описание,Расположение в парке,Форма осмотра,global_id,NaN
1,1,Ель обыкновенная,Picea abies,ba4bd41f-4301-421c-892b-f9b74da0b8de,Хвойный лес,Май\n,Ботаники выделяют 40 видов елей. Ель обыкновен...,Хвойный лес,Бесплатная,881409012,NaN
2,2,Сосна обыкновенная,Pinus sylvestris,102b0915-249e-4c3b-8912-9e484a56db27,Хвойный лес,Май - июнь\n,В средней полосе России исконно растёт лишь од...,Хвойный лес,Бесплатная,881409317,NaN
3,3,Дуб черешчатый,Querqus robur,b0fd9656-54e1-4169-8255-db196c3b44c6,Смешанный лес,Май - июнь\n,Дуб черешчатый считается визитной карточкой ши...,Смешанный лес,Бесплатная,881409921,NaN
4,4,Липа мелколистная,Tilia cordata,eadb690c-3c4c-4878-9b93-1cc032938807,Смешанный лес,Июль\n,В Европейской части России можно встретить чет...,Смешанный лес,Бесплатная,881410524,NaN


## Вывод сводного отчета по тестированию. Создание дополнительных тестовых случаев, которые включают опечатки и нерелевантные слова. Расчет Accuracy.

In [18]:

def create_test_cases(plant_list):
    """Создание структурированного набора тестовых случаев"""
    non_plant_words = [
        'париж', 'слон', 'самолет', 'зона', 'климат', 
        'цветение', 'шоколад', 'европа', 'компьютер'
    ]
     # Создаем словарь с тестовыми случаями
    test_cases = {
        'exact_names': plant_list,
        'partial_names': [name.split()[0] for name in plant_list],
        'misspelled': [name.replace('о', 'а') for name in plant_list[:10]],
        'non_plant_queries': non_plant_words,
        'context_queries': [
            f"где растет {plant}" for plant in plant_list[:5]
        ]
    }
    return test_cases

def test_model_response(query, df_plants):
    """Тестирование модели на заданном запросе"""
    max_ratio = 0
    matched_name = None
     # Проходим по уникальным названиям растений в наборе данных
    for name in df_plants['Name'].unique():
        ratio = fuzz.partial_ratio(name.lower(), query.lower())
        if ratio > max_ratio and ratio > 70: # Если коэффициент больше максимального и превышает 70
            max_ratio = ratio
            matched_name = name
     # Возвращаем результат тестирования
    return {
        'query': query,
        'matched_name': matched_name,
        'confidence': max_ratio,
        'success': matched_name is not None
    }

def run_final_tests(df_plants):
    """Запуск финальных тестов на наборе данных растений"""
    plant_list = df_plants['Name'].unique().tolist()  # Получаем уникальные названия растений
    test_cases = create_test_cases(plant_list)  # Создаем тестовые случаи
    
    results = defaultdict(list) # Словарь для хранения результатов
    metrics = defaultdict(dict) # Словарь для хранения метрик
    unusual_responses = [] # Список для хранения неадекватных ответов
    
    # Выполнение тестов
    for category, queries in test_cases.items():
        if queries:  # Проверка на пустой список
            successful = 0
            total = len(queries)
            
            for query in queries: 
                result = test_model_response(query, df_plants) # Получаем результат тестирования
                results[category].append(result)
                
                if result['success']:
                    successful += 1
                    
                    # Проверка неадекватных ответов
                    if category == 'non_plant_queries' and result['confidence'] > 70:
                        unusual_responses.append({
                            'query': query,
                            'matched': result['matched_name'],
                            'confidence': result['confidence']
                        })
            
            if total > 0:  #  # Защита от деления на ноль
                metrics[category] = {
                    'total': total,
                    'successful': successful,
                    'accuracy': (successful/total)*100
                }
    
    return metrics, unusual_responses #Возвращаем метрики и необычные ответы

def print_final_report(metrics, unusual_responses):
    print("\nСВОДНЫЙ ОТЧЕТ ТЕСТИРОВАНИЯ")
    print("=" * 50)
    
    #  Общая метрика
    total_tests = sum(m['total'] for m in metrics.values())  # Общее количество тестов
    total_successful = sum(m['successful'] for m in metrics.values()) #Общее количество успешных совпадений
    overall_accuracy = (total_successful/total_tests)*100 if total_tests > 0 else 0 # accuracy
    
    print(f"\nОБЩАЯ ТОЧНОСТЬ: {overall_accuracy:.2f}%")
    print(f"Всего тестов: {total_tests}")
    print(f"Успешных совпадений: {total_successful}")
    
    print("\nМЕТРИКИ ПО КАТЕГОРИЯМ:")
    for category, metric in metrics.items():
        print(f"\n{category}:")
        print(f"Точность: {metric['accuracy']:.2f}%")
        print(f"Тестов: {metric['total']}")
        print(f"Успешных: {metric['successful']}")
    
    if unusual_responses:
        print("\nНЕАДЕКВАТНЫЕ ОТВЕТЫ НА НЕРЕЛЕВАНТНЫЕ ЗАПРОСЫ:")
        print("-" * 50)
        for resp in unusual_responses:
            print(f"Запрос: {resp['query']}")
            print(f"Ошибочное совпадение: {resp['matched']}")
            print(f"Уверенность: {resp['confidence']}%\n")

if __name__ == '__main__':
    # Загрузка данных
    CSV_PATH = r"C:\Users\Mikl\zaryadye_bot\60861CSV\data-60861-2024-08-06.csv"
    df_plants = pd.read_csv(CSV_PATH, sep=';', encoding='utf-8', on_bad_lines='skip')
    df_plants = df_plants[df_plants['ID'] != 'Код'].reset_index(drop=True)
    
    # Запуск тестирования
    metrics, unusual_responses = run_final_tests(df_plants)
    
    # Вывод отчета
    print_final_report(metrics, unusual_responses)




СВОДНЫЙ ОТЧЕТ ТЕСТИРОВАНИЯ

ОБЩАЯ ТОЧНОСТЬ: 99.10%
Всего тестов: 334
Успешных совпадений: 331

МЕТРИКИ ПО КАТЕГОРИЯМ:

exact_names:
Точность: 100.00%
Тестов: 155
Успешных: 155

partial_names:
Точность: 100.00%
Тестов: 155
Успешных: 155

misspelled:
Точность: 100.00%
Тестов: 10
Успешных: 10

non_plant_queries:
Точность: 66.67%
Тестов: 9
Успешных: 6

context_queries:
Точность: 100.00%
Тестов: 5
Успешных: 5

НЕАДЕКВАТНЫЕ ОТВЕТЫ НА НЕРЕЛЕВАНТНЫЕ ЗАПРОСЫ:
--------------------------------------------------
Запрос: париж
Ошибочное совпадение: Молочай кипарисовый
Уверенность: 80%

Запрос: слон
Ошибочное совпадение: Сосна обыкновенная
Уверенность: 75%

Запрос: зона
Ошибочное совпадение: Сосна обыкновенная
Уверенность: 75%

Запрос: цветение
Ошибочное совпадение: Ветреница дубравная
Уверенность: 75%

Запрос: шоколад
Ошибочное совпадение: Стрелолист широколистный
Уверенность: 71%

Запрос: европа
Ошибочное совпадение: Бересклет европейский
Уверенность: 83%



## Таким образом общая accurancy для нашей модели c учетом создания дополнительных тестовых случаев, которые включают опечатки и нерелевантные слова, составила 99.10%.